In [1]:
import pyrealsense2 as rs
import numpy as np
import matplotlib.pyplot as plt
from uarm.wrapper import SwiftAPI
import cv2
import imutils
from collections import deque
import time
import pid
%matplotlib qt

ModuleNotFoundError: No module named 'pyrealsense2'

In [4]:
swift = SwiftAPI(filters={'hwid': 'USB VID:PID=2341:0042'})

In [5]:
swift.waiting_ready(timeout=3)

In [6]:
device_info = swift.get_device_info()
print(device_info)
firmware_version = device_info['firmware_version']
if firmware_version and not firmware_version.startswith(('0.', '1.', '2.', '3.')):
    swift.set_speed_factor(0.0005)

{'api_version': '3.2.0', 'firmware_version': '3.2.0', 'device_unique': '50F14A43C6B1', 'device_type': None, 'hardware_version': None}


In [7]:
swift.set_mode(0)

0

In [8]:
swift.reset(wait=True, speed=1000)

In [9]:
swift.flush_cmd()

'OK'

In [10]:
swift.get_position()

[200.0, 0.0, 150.0]

In [11]:
FRAME_SIZE = [640, 480]

In [12]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, FRAME_SIZE[0], FRAME_SIZE[1], rs.format.z16, 30)
config.enable_stream(rs.stream.color, FRAME_SIZE[0], FRAME_SIZE[1], rs.format.bgr8, 30)


In [13]:
def mapObjectPosition(x, y):
    print("[INFO] obj. center coord. at x_0 = {0} | y_0 = {1}".format(x, y))

In [ ]:
def mapServoPosition(x, y, r):
    x_ = swift.get_position()[0]
    y_ = swift.get_position()[1]
    z_ = swift.get_position()[2]
    print(x_, y_, z_, r)
    
    y += 20
    
    if (r < 80):
        x_ += 10    
    
    if (x < 315):
        y_ += 2
    
    if (x > 325):
        y_ -= 2
        
    if (y < 235):
        z_ += 2
    
    if (y > 245):
        z_ -= 2

    swift.set_position(x=x_, y=y_, z=z_, speed=100)

In [ ]:
lower_range = np.array([0, 150, 150], dtype=np.uint8)
upper_range = np.array([64, 255, 255], dtype=np.uint8)

pts = deque(maxlen=100)

pipeline.start(config)
try:
    while True:
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

#         depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

#         images = np.hstack((color_image, depth_colormap))
        

        frame = imutils.resize(color_image, width=600)
        # frame = imutils.rotate(color_image, angle=180)
        
        hsv = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_range, upper_range)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        _, cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, 
                                cv2.CHAIN_APPROX_SIMPLE)
        center = None

        if len(cnts) > 0:
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            # print(M)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            center = (cx, cy)

            if radius > 10:
                # draw the circle and centroid on the frame,
                # then update the list of tracked points
                cv2.circle(frame, (int(x), int(y)), int(radius),
                    (0, 255, 255), 2)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)

                mapObjectPosition(int(x), int(y))
                depth = depth_frame.get_distance(int(x), int(y))
#                 print(depth)
                if not swift.get_is_moving():
                    mapServoPosition(x, y, radius)
                    swift.flush_cmd()
                
        pts.appendleft(center)

        for i in range(1, len(pts)):
            # if either of the tracked points are None, ignore
            # them
            if pts[i - 1] is None or pts[i] is None:
                continue
    
            # otherwise, compute the thickness of the line and
            # draw the connecting lines
            thickness = int(np.sqrt(100 / float(i + 1)) * 2.5)
            cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)

            cv2.imshow('Frame', frame)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord("q"):
                    break

    cv2.destroyAllWindows()
finally:
    pipeline.stop()
